In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd, os

# Reclame Aqui

In [4]:
bases = []
for company in os.listdir("../scraper/databases"):
    try:
        data = pd.read_csv(f"../scraper/databases/{company}/corpus.csv")
        data["company"] = company
        bases.append(data)
    except:
        print("NO DATA:", company)

database_rq = pd.concat(bases)

database_rq["source"] = "reclame_aqui"

print(f"COLUMNS: {database_rq.columns.to_list()}\nSAMPLES: {database_rq.shape}")

COLUMNS: ['id', 'title_ask', 'location_ask', 'date_ask', 'ask', 'date_answer', 'answer', 'date_final_answer', 'final_answer', 'business_again', 'service_note', 'resolved', 'link', 'company', 'source']
SAMPLES: (70259, 15)


In [5]:
database_rq[["resolved", "ask"]].groupby("resolved").count()

,ask
resolved,
Não resolvido,19649
Resolvido,50610


In [6]:
database_rq.head(2)

,id,title_ask,location_ask,date_ask,ask,date_answer,answer,date_final_answer,final_answer,business_again,service_note,resolved,link,company,source
0,187100053,Aplicativo de celular,-- - --,19/04/2024 às 08:45,Após a última atualização do aplicativo do Ban...,19/04/2024 às 14:17,"Olá, Leo.\n\nEsperamos que esteja bem!\n\nRece...",19/04/2024 às 14:29,Uma versão com problemas e a solução que dão é...,Não,0,Não resolvido,https://www.reclameaqui.com.br/itau/aplicativo...,itau,reclame_aqui
1,187093101,Redução de limites,Brasília - DF,19/04/2024 às 00:30,Venho por meio desta manifestar minha insatisf...,19/04/2024 às 13:51,"Olá, Leandro.\n\nEsperamos que esteja bem!\n\n...",20/04/2024 às 00:15,"Pessimo atendimento, nem ao menos tiveram a co...",Não,0,Não resolvido,https://www.reclameaqui.com.br/itau/reducao-de...,itau,reclame_aqui


# Consumidor.gov

In [7]:
path = "G:/.shortcut-targets-by-id/1pQB43wIngf-SaNu7oKBv_cfgHgc6vyaN/Iniciacao2024"

files = [i for i in os.listdir(path) if ".csv" in i]
database_cg_ = []
for f in files:
    df = pd.read_csv(f"{path}/{f}", sep=",")
    df["company"] = f.replace(".csv", "")
    database_cg_.append(df)

In [8]:
database_cg = pd.concat(database_cg_)

database_cg = database_cg.rename(columns={
    'date': 'date_ask', 
    'city': 'location_ask', 
    'state': 'uf_ask', 
    'isResolved': 'resolved', 
    'complain': 'ask', 
    'response': 'answer',
    'evaluation': 'service_note', 
    'evaluationText': 'final_answer', 
})

database_cg["business_again"] = "Nada"
database_cg["source"] = "consumidor_gov"

In [9]:
database_cg["resolved"] = ["Não resolvido" if d != "Resolvido" else "Resolvido" for d in database_cg["resolved"]]
database_cg[["resolved", "ask"]].groupby("resolved").count()

,ask
resolved,
Não resolvido,781067
Resolvido,815933


In [10]:
notes = []
errors = 0
for i in database_cg["service_note"].str.replace("Nota ", ""):
    try:
        notes.append(int(i)*2)
    except:
        notes.append(-1)
        errors += 1

database_cg["service_note"] = notes
database_cg = database_cg[database_cg["service_note"] != -1]

print(f"NOT FOUND: {errors}")
print(f"COLUMNS: {database_cg.columns.to_list()}\nSAMPLES: {database_cg.shape}")

NOT FOUND: 619697
COLUMNS: ['status', 'date_ask', 'location_ask', 'uf_ask', 'resolved', 'ask', 'answer', 'service_note', 'final_answer', 'company', 'business_again', 'source']
SAMPLES: (977303, 12)


In [11]:
database_cg.head(3)

,status,date_ask,location_ask,uf_ask,resolved,ask,answer,service_note,final_answer,company,business_again,source
0,Resolvido,19/08/2023,São Paulo,SP,Resolvido,É impressionante a capacidade dessa empresa tr...,"""Prezado(a) Sr(a). LUCAS, em atenção à sua rec...",10,Jhonatan muito ágil e solicito. Apesar do tran...,claroFixo,Nada,consumidor_gov
1,Resolvido,16/08/2023,Recife,PE,Resolvido,XXX.XXX.XXX-XX2866 Protocolo de cancelamento ...,"""Prezado(a) Sr(a). PRISCILLA, em atenção à sua...",8,<não há comentários do consumidor>,claroFixo,Nada,consumidor_gov
2,Resolvido,14/08/2023,Tatuí,SP,Resolvido,Em 04/08/2023 efetuei o pagamento da fatura do...,"""Prezado(a) Sr(a). MARIA, em atenção à sua rec...",8,<não há comentários do consumidor>,claroFixo,Nada,consumidor_gov


# Salving

In [12]:
reclamacoes = pd.concat([database_rq, database_cg])
reclamacoes[["resolved", "ask"]].groupby("resolved").count()

,ask
resolved,
Não resolvido,181019
Resolvido,866543


In [13]:
reclamacoes.head(3)

,id,title_ask,location_ask,date_ask,ask,date_answer,answer,date_final_answer,final_answer,business_again,service_note,resolved,link,company,source,status,uf_ask
0,187100053.0,Aplicativo de celular,-- - --,19/04/2024 às 08:45,Após a última atualização do aplicativo do Ban...,19/04/2024 às 14:17,"Olá, Leo.\n\nEsperamos que esteja bem!\n\nRece...",19/04/2024 às 14:29,Uma versão com problemas e a solução que dão é...,Não,0,Não resolvido,https://www.reclameaqui.com.br/itau/aplicativo...,itau,reclame_aqui,NaN,NaN
1,187093101.0,Redução de limites,Brasília - DF,19/04/2024 às 00:30,Venho por meio desta manifestar minha insatisf...,19/04/2024 às 13:51,"Olá, Leandro.\n\nEsperamos que esteja bem!\n\n...",20/04/2024 às 00:15,"Pessimo atendimento, nem ao menos tiveram a co...",Não,0,Não resolvido,https://www.reclameaqui.com.br/itau/reducao-de...,itau,reclame_aqui,NaN,NaN
2,187092297.0,Demora pra atendimento Uniclass,São Paulo - SP,18/04/2024 às 23:56,Hoje dia 18/04/24 estive na minha agência no p...,19/04/2024 às 11:34,"Olá, Jorge esperamos que esteja bem.\n\nInicia...",19/04/2024 às 18:18,Esperava que o gerente geral da agência reclam...,Sim,1,Não resolvido,https://www.reclameaqui.com.br/itau/demora-pra...,itau,reclame_aqui,NaN,NaN


In [14]:
reclamacoes.to_parquet("db_reclamacoes.parquet", index=False)